In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/서울월세_dataset.csv')
df.head()

,구분,질문,답변,embedding
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,...","[-0.63754356, -0.5960052, 0.5261206, -1.097488..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요,"[-0.580972, -0.23177795, 0.4248164, -0.6200277..."
2,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...,"[0.18744174, 0.18414854, 0.24230862, -0.493406..."
3,월세,서울월세지원 10번 받아서 끝났는데 또 신청해서 또 받을수 있나요,답변드립니다최대 10개월 지원받고 200만원까지 가능한데 생애 1회만 가능합니다그러...,"[-0.5501612, -0.4678361, 0.82305604, -0.490082..."
4,월세,"서울시에서 하는 서울 월세지원에 선정되었는데4.5,6,7,8월 금액이 9월 말에 d...",괜찮습니다,"[-0.23974398, -0.34466678, 0.14065771, -0.7792..."


In [3]:
df.drop(columns=['구분','embedding'], inplace=True)

- 데이터 전처리

In [4]:
# 결측치 확인
df.isna().sum().sum()

0

In [5]:
# 중복 데이터 확인
df.shape, df.답변.nunique()

((257, 2), 211)

In [6]:
# 중복 데이터 제거
df.drop_duplicates(subset=['답변'], inplace=True)
df.shape

(211, 2)

In [7]:
# 한글 이외 데이터 삭제
df.답변 = df.답변.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()

In [8]:
# '' 제거
df.답변.replace('', np.nan, inplace=True)
df.답변.isna().sum()

0

In [9]:
df.to_csv('data/서울월세 전처리완료.tsv', sep='\t', index=False)

- 한글 불용어 처리

In [10]:
with open('data/한글불용어100.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [11]:
from konlpy.tag import Okt
okt = Okt()

In [12]:
from tqdm import tqdm
answers = []
for 답변 in tqdm(df.답변):
    morphs = okt.morphs(답변, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    answers.append(clean_morph_review)

100%|██████████| 211/211 [00:08<00:00, 26.08it/s]


In [13]:
df['processed'] = answers
df.to_csv('data/서울월세 형태소처리완료.tsv', index=False)

- 데이터셋 분리

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    answers, df.score.values, stratify=df.score.values, random_state=2023
)

AttributeError: 'DataFrame' object has no attribute 'score'